In [1]:
import pandas as pd
import numpy as np

In [2]:
DIR = '../dataset/'
DATASET = 'Nutrition_Physical_Activity_and_Obesity_Data.csv'
df = pd.read_csv(DIR + DATASET)

/tmp/ipykernel_5016/1955201767.py:3: DtypeWarning: Columns (18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DIR + DATASET)


In [3]:
df.columns = df.columns.str.strip()

for col in df.columns:
    if df[col].dtype != np.dtypes.Int64DType:
        continue

    df[col] = df[col].str.rstrip()
    df[col] = df[col].str.rstrip()

df['Data_Value'] = df['Data_Value'].replace('', -1)
df['Data_Value'] = df['Data_Value'].astype(float)

In [4]:
states = df['LocationDesc'].unique()
years = list(range(2011, 2023))*len(states)
states = np.repeat(states, 12)
new_df = pd.DataFrame({'state': states, 'year': years})
new_df

,state,year
0,Alaska,2011
1,Alaska,2012
2,Alaska,2013
3,Alaska,2014
4,Alaska,2015
...,...,...
655,Oregon,2018
656,Oregon,2019
657,Oregon,2020
658,Oregon,2021


In [5]:
#df.to_csv('../udataset/' + DATASET, index=False)

def get_label_function(desired_question, df, desired_col):

    def get_attr_column(row):
        to_return = df[(df['Question'] == desired_question) & (df['YearEnd'] == row['year']) & (df['LocationDesc'] == row['state']) & (df['Data_Value'] != -1) & (df['StratificationCategory1'] == 'Total')][desired_col].mean()

        return to_return

    return get_attr_column

In [6]:
desired_questions = ['Percent of adults aged 18 years and older who have obesity', 'Percent of adults aged 18 years and older who have an overweight classification', 'Percent of adults who engage in no leisure-time physical activity']
column_names = ['obesity_adult_rate', 'overweight_adult_rate', 'adults_no_leisure-time_physical_activity']

In [7]:
row = df[(df['Question'] == 'Percent of adults aged 18 years and older who have obesity') & (df['YearEnd'] == 2012) & (df['LocationDesc'] == 'Alaska') & (df['Data_Value'] != -1)]['Data_Value']
print(row.mean())
for index, question in enumerate(desired_questions):
    new_df[column_names[index]] = new_df.apply(get_label_function(question, df, 'Data_Value'), axis=1)

24.929629629629627


In [8]:
new_df

,state,year,obesity_adult_rate,overweight_adult_rate,adults_no_leisure-time_physical_activity
0,Alaska,2011,27.4,38.9,22.0
1,Alaska,2012,25.7,39.1,18.5
2,Alaska,2013,28.4,37.7,22.3
3,Alaska,2014,29.7,35.1,19.2
4,Alaska,2015,29.8,37.4,22.0
...,...,...,...,...,...
655,Oregon,2018,29.9,34.2,19.3
656,Oregon,2019,29.0,34.9,23.9
657,Oregon,2020,28.1,36.2,18.5
658,Oregon,2021,30.4,34.7,19.9


In [9]:
for col in new_df.columns[2:]:
    new_df[col] = (new_df[col] - new_df[col].min())/(new_df[col].max() - new_df[col].min())

In [10]:
new_df = new_df[new_df['state'] !='Virgin Islands']

for col in new_df.columns:
    for state in new_df[pd.isna(new_df[col])]['state'].unique():
        new_df.loc[new_df['state'] == state, col] = new_df[new_df['state'] == state][col].fillna(new_df[(new_df['state'] == state) & (pd.notna(new_df[col]))][col].mean())


In [11]:
NEWNAME = 'health_data.csv'
new_df.to_csv('../udataset/' + NEWNAME, index=False)